In [4]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/RawData.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [5]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st19-15-19/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [19, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 19], [19], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '15'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 19]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st19-15-19/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,19)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.260183539
Validation Loss: 2.8845406
Epoch: 0002 cost = 2.241298267
Validation Loss: 2.6508455
Epoch: 0003 cost = 2.223184041
Validation Loss: 2.339541
Epoch: 0004 cost = 2.205044133
Validation Loss: 2.171406
Epoch: 0005 cost = 2.185960906
Validation Loss: 2.1097667
Epoch: 0006 cost = 2.164894955
Validation Loss: 2.0566497
Epoch: 0007 cost = 2.141316244
Validation Loss: 2.0342026
Epoch: 0008 cost = 2.116648742
Validation Loss: 2.0022588
Epoch: 0009 cost = 2.093047244
Validation Loss: 1.980436
Epoch: 0010 cost = 2.071268184
Validation Loss: 1.9766514
Epoch: 0011 cost = 2.051053081
Validation Loss: 1.9592553
Epoch: 0012 cost = 2.032597780
Validation Loss: 1.9445931
Epoch: 0013 cost = 2.016433784
Validation Loss: 1.922557
Epoch: 0014 cost = 2.002447162
Validation Loss: 1.9207329
Epoch: 0015 cost = 1.990024362
Validation Loss: 1.9288937
Epoch: 0016 cost = 1.978590659
Validation Loss: 1.9388604
Epoch: 0017 cost = 1.967752576
Validation Loss: 1.9466479
Epoch: 00

Validation Loss: 0.978444
Epoch: 0147 cost = 1.014627542
Validation Loss: 0.9520143
Epoch: 0148 cost = 1.009771705
Validation Loss: 0.9751883
Epoch: 0149 cost = 1.004940552
Validation Loss: 0.94802904
Epoch: 0150 cost = 1.000134494
Validation Loss: 0.9448342
Epoch: 0151 cost = 0.995352600
Validation Loss: 0.9834886
Epoch: 0152 cost = 0.990595332
Validation Loss: 0.9905404
Epoch: 0153 cost = 0.985862587
Validation Loss: 0.9619357
Epoch: 0154 cost = 0.981154101
Validation Loss: 0.9206453
Epoch: 0155 cost = 0.976469823
Validation Loss: 0.92438936
Epoch: 0156 cost = 0.971809890
Validation Loss: 0.9184471
Epoch: 0157 cost = 0.967174028
Validation Loss: 0.9066758
Epoch: 0158 cost = 0.962562263
Validation Loss: 0.91955197
Epoch: 0159 cost = 0.957974391
Validation Loss: 0.8859887
Epoch: 0160 cost = 0.953410498
Validation Loss: 0.8811079
Epoch: 0161 cost = 0.948870608
Validation Loss: 0.9038403
Epoch: 0162 cost = 0.944354662
Validation Loss: 0.89100426
Epoch: 0163 cost = 0.939862302
Validation 

Epoch: 0289 cost = 0.535839311
Validation Loss: 0.48570815
Epoch: 0290 cost = 0.533688528
Validation Loss: 0.45827594
Epoch: 0291 cost = 0.531552374
Validation Loss: 0.46786976
Epoch: 0292 cost = 0.529429649
Validation Loss: 0.48067415
Epoch: 0293 cost = 0.527319508
Validation Loss: 0.48661083
Epoch: 0294 cost = 0.525220352
Validation Loss: 0.47834203
Epoch: 0295 cost = 0.523130485
Validation Loss: 0.4788713
Epoch: 0296 cost = 0.521047073
Validation Loss: 0.49568248
Epoch: 0297 cost = 0.518966496
Validation Loss: 0.47527137
Epoch: 0298 cost = 0.516884433
Validation Loss: 0.48032498
Epoch: 0299 cost = 0.514795406
Validation Loss: 0.48877504
Epoch: 0300 cost = 0.512695743
Validation Loss: 0.47526738
Epoch: 0301 cost = 0.510585338
Validation Loss: 0.43280238
Epoch: 0302 cost = 0.508470425
Validation Loss: 0.47554088
Epoch: 0303 cost = 0.506362230
Validation Loss: 0.47365826
Epoch: 0304 cost = 0.504270762
Validation Loss: 0.48408005
Epoch: 0305 cost = 0.502201898
Validation Loss: 0.4611912

Epoch: 0435 cost = 0.320974525
Validation Loss: 0.2707377
Epoch: 0436 cost = 0.320040230
Validation Loss: 0.27776787
Epoch: 0437 cost = 0.319111011
Validation Loss: 0.29835814
Epoch: 0438 cost = 0.318186773
Validation Loss: 0.31544966
Epoch: 0439 cost = 0.317267541
Validation Loss: 0.29758796
Epoch: 0440 cost = 0.316353283
Validation Loss: 0.3080618
Epoch: 0441 cost = 0.315443920
Validation Loss: 0.30051696
Epoch: 0442 cost = 0.314539386
Validation Loss: 0.3077827
Epoch: 0443 cost = 0.313639747
Validation Loss: 0.3108242
Epoch: 0444 cost = 0.312744958
Validation Loss: 0.31433478
Epoch: 0445 cost = 0.311854912
Validation Loss: 0.30233476
Epoch: 0446 cost = 0.310969817
Validation Loss: 0.3200366
Epoch: 0447 cost = 0.310089282
Validation Loss: 0.32951725
Epoch: 0448 cost = 0.309213553
Validation Loss: 0.31874648
Epoch: 0449 cost = 0.308342423
Validation Loss: 0.3195128
Epoch: 0450 cost = 0.307475984
Validation Loss: 0.30110955
Epoch: 0451 cost = 0.306614203
Validation Loss: 0.28281435
Epo

Epoch: 0586 cost = 0.223726973
Validation Loss: 0.18599658
Epoch: 0587 cost = 0.223310705
Validation Loss: 0.19139326
Epoch: 0588 cost = 0.222896814
Validation Loss: 0.17916645
Epoch: 0589 cost = 0.222485263
Validation Loss: 0.18076463
Epoch: 0590 cost = 0.222076163
Validation Loss: 0.1817307
Epoch: 0591 cost = 0.221669374
Validation Loss: 0.18474367
Epoch: 0592 cost = 0.221264907
Validation Loss: 0.18909556
Epoch: 0593 cost = 0.220862804
Validation Loss: 0.18010564
Epoch: 0594 cost = 0.220463023
Validation Loss: 0.17764336
Epoch: 0595 cost = 0.220065604
Validation Loss: 0.16938068
Epoch: 0596 cost = 0.219670481
Validation Loss: 0.17266963
Epoch: 0597 cost = 0.219277639
Validation Loss: 0.17213428
Epoch: 0598 cost = 0.218887161
Validation Loss: 0.17687261
Epoch: 0599 cost = 0.218498909
Validation Loss: 0.16766547
Epoch: 0600 cost = 0.218113016
Validation Loss: 0.17029803
Epoch: 0601 cost = 0.217729347
Validation Loss: 0.17951643
Epoch: 0602 cost = 0.217347967
Validation Loss: 0.1853189

...

Validation Loss: 0.032921206
Epoch: 4374 cost = 0.025844889
Validation Loss: 0.033148736
Epoch: 4375 cost = 0.025844553
Validation Loss: 0.032928955
Epoch: 4376 cost = 0.025844153
Validation Loss: 0.037331995
Epoch: 4377 cost = 0.025843781
Validation Loss: 0.040223982
Epoch: 4378 cost = 0.025843425
Validation Loss: 0.04065429
Epoch: 4379 cost = 0.025843021
Validation Loss: 0.033728156
Epoch: 4380 cost = 0.025842632
Validation Loss: 0.036775157
Epoch: 4381 cost = 0.025842278
Validation Loss: 0.032001633
Epoch: 4382 cost = 0.025841891
Validation Loss: 0.0337992
Epoch: 4383 cost = 0.025841569
Validation Loss: 0.036199972
Epoch: 4384 cost = 0.025841187
Validation Loss: 0.03033897
Epoch: 4385 cost = 0.025840808
Validation Loss: 0.03249267
Epoch: 4386 cost = 0.025840420
Validation Loss: 0.03181469
Epoch: 4387 cost = 0.025840045
Validation Loss: 0.040889934
Epoch: 4388 cost = 0.025839703
Validation Loss: 0.03707829
Epoch: 4389 cost = 0.025839316
Validation Loss: 0.041688766
Epoch: 4390 cost =

Validation Loss: 0.067619145
Epoch: 4517 cost = 0.025794060
Validation Loss: 0.061465126
Epoch: 4518 cost = 0.025793677
Validation Loss: 0.05733314
Epoch: 4519 cost = 0.025793307
Validation Loss: 0.04125661
Epoch: 4520 cost = 0.025793028
Validation Loss: 0.03972413
Epoch: 4521 cost = 0.025792658
Validation Loss: 0.063018665
Epoch: 4522 cost = 0.025792281
Validation Loss: 0.05328435
Epoch: 4523 cost = 0.025791972
Validation Loss: 0.075515226
Epoch: 4524 cost = 0.025791662
Validation Loss: 0.05886096
Epoch: 4525 cost = 0.025791275
Validation Loss: 0.045392267
Epoch: 4526 cost = 0.025790990
Validation Loss: 0.03161478
Epoch: 4527 cost = 0.025790657
Validation Loss: 0.03516045
Epoch: 4528 cost = 0.025790288
Validation Loss: 0.040730443
Epoch: 4529 cost = 0.025789996
Validation Loss: 0.038817596
Epoch: 4530 cost = 0.025789618
Validation Loss: 0.037158713
Epoch: 4531 cost = 0.025789291
Validation Loss: 0.043075692
Epoch: 4532 cost = 0.025788979
Validation Loss: 0.047168996
Epoch: 4533 cost =

Validation Loss: 0.03980103
Epoch: 4661 cost = 0.025747353
Validation Loss: 0.03676546
Epoch: 4662 cost = 0.025747130
Validation Loss: 0.038810495
Epoch: 4663 cost = 0.025746796
Validation Loss: 0.051281303
Epoch: 4664 cost = 0.025746520
Validation Loss: 0.073111705
Epoch: 4665 cost = 0.025746180
Validation Loss: 0.0660823
Epoch: 4666 cost = 0.025745900
Validation Loss: 0.058851197
Epoch: 4667 cost = 0.025745587
Validation Loss: 0.06137714
Epoch: 4668 cost = 0.025745306
Validation Loss: 0.052876845
Epoch: 4669 cost = 0.025744975
Validation Loss: 0.035220787
Epoch: 4670 cost = 0.025744693
Validation Loss: 0.033215486
Epoch: 4671 cost = 0.025744379
Validation Loss: 0.037169125
Epoch: 4672 cost = 0.025744059
Validation Loss: 0.037510097
Epoch: 4673 cost = 0.025743835
Validation Loss: 0.042206973
Epoch: 4674 cost = 0.025743490
Validation Loss: 0.041631542
Epoch: 4675 cost = 0.025743182
Validation Loss: 0.041677304
Epoch: 4676 cost = 0.025742911
Validation Loss: 0.03666253
Epoch: 4677 cost 

Validation Loss: 0.031354666
Epoch: 4809 cost = 0.025701035
Validation Loss: 0.0325366
Epoch: 4810 cost = 0.025700715
Validation Loss: 0.03396839
Epoch: 4811 cost = 0.025700422
Validation Loss: 0.035454873
Epoch: 4812 cost = 0.025700159
Validation Loss: 0.033352952
Epoch: 4813 cost = 0.025699887
Validation Loss: 0.03148126
Epoch: 4814 cost = 0.025699596
Validation Loss: 0.03362192
Epoch: 4815 cost = 0.025699294
Validation Loss: 0.03322642
Epoch: 4816 cost = 0.025699008
Validation Loss: 0.031090552
Epoch: 4817 cost = 0.025698744
Validation Loss: 0.029092768
Epoch: 4818 cost = 0.025698432
Validation Loss: 0.0298952
Epoch: 4819 cost = 0.025698222
Validation Loss: 0.03209471
Epoch: 4820 cost = 0.025697878
Validation Loss: 0.03435915
Epoch: 4821 cost = 0.025697596
Validation Loss: 0.032456942
Epoch: 4822 cost = 0.025697362
Validation Loss: 0.031812042
Epoch: 4823 cost = 0.025697055
Validation Loss: 0.029467853
Epoch: 4824 cost = 0.025696802
Validation Loss: 0.034414213
Epoch: 4825 cost = 0.

Epoch: 4949 cost = 0.025661662
Validation Loss: 0.045727447
Epoch: 4950 cost = 0.025661345
Validation Loss: 0.035009094
Epoch: 4951 cost = 0.025661090
Validation Loss: 0.032567747
Epoch: 4952 cost = 0.025660848
Validation Loss: 0.035573073
Epoch: 4953 cost = 0.025660504
Validation Loss: 0.035532095
Epoch: 4954 cost = 0.025660236
Validation Loss: 0.03710632
Epoch: 4955 cost = 0.025659948
Validation Loss: 0.039207585
Epoch: 4956 cost = 0.025659653
Validation Loss: 0.03518027
Epoch: 4957 cost = 0.025659388
Validation Loss: 0.03187885
Epoch: 4958 cost = 0.025659108
Validation Loss: 0.036987577
Epoch: 4959 cost = 0.025658814
Validation Loss: 0.032920342
Epoch: 4960 cost = 0.025658572
Validation Loss: 0.037252046
Epoch: 4961 cost = 0.025658301
Validation Loss: 0.039110024
Epoch: 4962 cost = 0.025657991
Validation Loss: 0.034507435
Epoch: 4963 cost = 0.025657697
Validation Loss: 0.0392993
Epoch: 4964 cost = 0.025657396
Validation Loss: 0.039733857
Epoch: 4965 cost = 0.025657117
Validation Los

In [6]:
currentmin

0.027774975

In [7]:
togive2

[array([[0.01276377, 0.8053072 , 0.42155513, 0.5055225 , 0.7092401 ,
         0.94296   , 0.283776  , 0.4910083 , 0.9192298 , 0.95906043,
         0.04284439, 0.6582974 , 0.386457  , 0.8570693 , 0.50587106]],
       dtype=float32),
 array([[0.00948077, 0.8023985 , 0.44545788, 0.52687234, 0.71887267,
         0.9568662 , 0.2899333 , 0.50239754, 0.929103  , 0.97104275,
         0.03338652, 0.6911974 , 0.4010063 , 0.8324401 , 0.525347  ]],
       dtype=float32),
 array([[0.01390988, 0.7622969 , 0.42391008, 0.46666253, 0.7324156 ,
         0.94664097, 0.4167286 , 0.5462175 , 0.9072104 , 0.95939344,
         0.03864054, 0.62672895, 0.49858832, 0.8072751 , 0.56141484]],
       dtype=float32),
 array([[0.01081361, 0.7978565 , 0.412372  , 0.47122818, 0.73125196,
         0.9564731 , 0.4608774 , 0.5401551 , 0.93310535, 0.9689599 ,
         0.03153424, 0.45797515, 0.5257955 , 0.8137742 , 0.54287237]],
       dtype=float32),
 array([[0.01727001, 0.78246176, 0.37758967, 0.4010683 , 0.7333381 ,
   

In [8]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult19-15-19.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))